In [8]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, binomtest, chi2_contingency

In [21]:
df = pd.read_excel('../data/interim/masterdbForNLP.xlsx')
df = df.replace('Unknown', np.nan)
df.loc[:, 'EPA'] = df['EPA'].str[0].str.upper().replace('D',np.nan)
df.loc[:, 'Type'] = df['Type']
# df.loc[:, 'PGY'] = df['PGY'].replace({'Unknown': np.nan})

In [22]:
df.columns

Index(['Unnamed: 0', 'Survey N', 'Question N', 'commentId', 'dataSource',
       'NumberFromRawData', 'comment', 'rating', 'q1p1', 'q1p2', 'q2p1',
       'q2p2', 'q3p1', 'q3p2', 'q1p1T', 'q1p2T', 'q2p1T', 'q2p2T', 'q3p1T',
       'q3p2T', 'P1QualScore', 'P2QualScore', 'Q1Match', 'Q2Match', 'Q3Match',
       'perfectMatch', 'RobMacCommentModified', 'RobMacQ1', 'RobMacQ2',
       'RobMacQ3', 'RobMacQualScore', 'Q1', 'Q2', 'Q3', 'QUAL', 'GenderRes',
       'GenderFac', 'Type', 'ObserverType', 'EPA', 'PGY', 'Q1c', 'Q2i', 'Q3i',
       'QUALc'],
      dtype='object')

In [23]:
def get_table(col : str, gb='dataSource', func='value_counts', func_args : dict={'normalize':True}, rename='Frequency') -> pd.DataFrame:
    res = pd.DataFrame(df.groupby(gb)[col].agg(func, **func_args))
    if rename is not None:
        res = res.rename({col:rename}, axis=1)
    res = res.reset_index().pivot(index=col, columns=gb)
    return res

In [24]:
cols_to_compare = ['rating','QUAL','GenderRes','GenderFac','Type','ObserverType','EPA','PGY']
for c in cols_to_compare:
    print(c)
    tbl = get_table(c, func_args={'normalize':False})
    print(tbl)
    _, p, _, _ = chi2_contingency(tbl)
    print('PVal', p)
    print()

rating
           Frequency     
dataSource       Mac  Sas
rating                   
1.0                2    2
2.0               15   32
3.0              110  186
4.0              509  431
5.0              612  599
PVal 1.6825871016678218e-06

QUAL
           Frequency     
dataSource       Mac  Sas
QUAL                     
0                 91    9
1                228  110
2                404  281
3                434  523
4                 34   43
5                 59  284
PVal 5.003638371971221e-60

GenderRes
           Frequency     
dataSource       Mac  Sas
GenderRes                
Female           740  297
Male             510  953
PVal 5.501826327438153e-72

GenderFac
           Frequency     
dataSource       Mac  Sas
GenderFac                
Female           487  390
Male             748  699
PVal 0.07944161932344407

Type
                                                   Frequency      
dataSource                                               Mac   Sas
Type            

In [8]:
cols_to_compare_quant = ['rating','QUAL','PGY']
for c in cols_to_compare_quant:
    print(c)
    print(df.groupby('dataSource')[c].mean())
    _, pval = ttest_ind(df.loc[df['dataSource'] == 'Mac', c], df.loc[df['dataSource'] == 'Sas', c], 
                        alternative='two-sided', nan_policy='omit')
    print('PVal', pval)
    print()

rating
dataSource
Mac    4.373397
Sas    4.274400
Name: rating, dtype: float64
PVal 0.0012473072643488876

QUAL
dataSource
Mac    2.2152
Sas    3.0664
Name: QUAL, dtype: float64
PVal 3.860445378176051e-66

PGY
dataSource
Mac    1.969576
Sas         NaN
Name: PGY, dtype: float64
PVal --



/home/maxspad/mambaforge/envs/nlp-qual-max/lib/python3.9/site-packages/scipy/stats/_mstats_basic.py:1279: RuntimeWarning: divide by zero encountered in true_divide
  denom = ma.sqrt(svar*(1.0/n1 + 1.0/n2))  # n-D computation here!


In [ ]:
for c in cols_to_compare_quant